In [1]:
import numpy as np

from goForwardGame import vectorizedGoForwardGamesTest
from ga_neural_net import ANN

vectorizedGame = vectorizedGoForwardGamesTest()

In [2]:
vectorizedGame.reset()

In [3]:
players = []
for _ in range(4):
    players.append(ANN(layers_unit=[732, 128, 128, 1]))

gen = 1

In [4]:
EPOCH_GAME = 1 # Number of games before select best to crossover
EPOCH = 2

logfile = open('log.csv', 'a')

for _ in range(EPOCH):
    scoreboard = np.zeros(4)
    for game_index in range(EPOCH_GAME):
        vectorizedGame.reset()
        is_done = False
        game_steps_history = []

        logfile.write(f'{gen},{game_index+1},')
        print(f'gen {gen}, game {game_index+1}')
        # Play a game
        while not is_done:
            _, string_cards, actions, number_actions = vectorizedGame.info()
            current_player, state, action_space = vectorizedGame.getCurrentState()
            step = min(int(players[current_player-1].get_step(state[0]) * len(number_actions)), len(number_actions)-1)
            rewards, is_done, end_info = vectorizedGame.step(number_actions[step])
            game_steps_history.append(number_actions[step])

        scoreboard += rewards
        logfile.write(f'"{str(game_steps_history)}","{rewards}"\n')
    print('After game' ,game_index, 'scoreboard:', scoreboard/EPOCH_GAME)
    
    rank = scoreboard.argsort()[::-1]
    players[rank[2]] = players[rank[0]].fuck_with(players[rank[1]], 0.5)
    players[rank[3]] = players[rank[0]].fuck_with(players[rank[1]], 0.1)
    gen+=1

logfile.close()

gen 1, game 1
After game 0 scoreboard: [ 19. -13.  -2.  -4.]
gen 2, game 1
After game 0 scoreboard: [ 29. -13.  -3. -13.]


In [5]:
# Export model
players[0].export_model('model.json')

In [6]:
# Load model from saved file
model = ANN.load_model('model.json')

732
128
128
1


In [7]:
model.get_step(state[0])

array([0.20632162])